In [1]:
import os
import streamlit as st
import pickle
import time
from dotenv import load_dotenv
import langchain
from langchain import LLMChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import glob

import google.generativeai as genai
from google.generativeai import GenerativeModel
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

In [12]:
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [13]:
genai.configure()

In [14]:
MODEL="gemini-2.0-flash-lite"

In [5]:
llm = ChatGoogleGenerativeAI(model=MODEL, temperature=0.5)

In [9]:
file_path = r'../Database/Boeing (BA) Q1 2024.pdf'

In [16]:
CUSTOM_PROMPT = "Write a detailed summary of the methodology used in following paper. Give output in markdown format."
vectorDB_path = 'faiss_store'

In [17]:
loader = PyPDFLoader(file_path)
docs = loader.load_and_split()
prompt_template = CUSTOM_PROMPT + """
Answer the following question based only on the provided context, do not use any external information.:

<context>
{text}
</context>
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=['text'])
llm = ChatGoogleGenerativeAI(model=MODEL, temperature=0.5)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(docs, embeddings)
vector_store.save_local(vectorDB_path)
vectorstore = FAISS.load_local(vectorDB_path, embeddings, allow_dangerous_deserialization=True)  # Load the FAISS index
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
result = chain({"question": prompt_template}, return_only_outputs=True)

/var/folders/n_/tb_qk64d4q740jlrg8457tvc0000gn/T/ipykernel_4768/3571785204.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": prompt_template}, return_only_outputs=True)


: 

In [ ]:
# CUSTOM_PROMPT = "Write a concise summary of the following paper with this structure: Problem being solved, Approach, Main results and Main Discussion Points. Give output in markdown format."
CUSTOM_PROMPT = "Write a concise summary of the methodology used in following paper. Give output in markdown format."
custom_summaries = custom_summary(file_path, MODEL, custom_prompt=CUSTOM_PROMPT)